In [1]:
from collections import namedtuple

In [2]:
import os
os.chdir("../")

In [3]:
DataIngestionConfig = namedtuple('DataIngestionConfig', [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [4]:
# current path
print(os.getcwd())

c:\Users\nitis\Code\INEURON\INEURON_Implentation\CLASS_Github_Template\Deep_cnn_classifier


In [5]:
os.chdir(r"C:\Users\nitis\Code\INEURON\INEURON_Implentation\CLASS_Github_Template\Deep_cnn_classifier\src")

In [22]:
from  deepClassifier.constants import *
from deepClassifier.utils.common import read_yaml,create_directories
import stat
import pathlib as path


In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [23]:
import urllib.request as request
from zipfile import ZipFile

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers= request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
    def _get_updated_list_of_files(self, list_of_files):
        return [file for file in list_of_files if not file.endswith(".jpg") and ("Cat" in file or "Dog" in file)]
    
    def _preprocess(self,zf,f,working_dir:str):
        target_filepath = os.path.join(working_dir,f)
        if not os.path.exists(target_filepath):
            zf.extract(f,working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            #os.chmod(target_filepath, stat.S_IWRITE)
            os.remove(target_filepath)


    def unzip_and_clean(self):
        with ZipFile(self.config.local_data_file,mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf,f,self.config.unzip_dir)
                

    # def save_data(self):
    #     pass

    # def run(self):
    #     self.download_data()
    #     self.unzip_data()
    #     self.save_data()

In [9]:
os.chdir("../")

In [24]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e


[2022-10-16 22:09:05,675: INFO: common] yaml file: configs\config.yaml loaded successfully
[2022-10-16 22:09:05,676: INFO: common] yaml file: params.yaml loaded successfully
[2022-10-16 22:09:05,681: INFO: common] created directory at: artifacts
[2022-10-16 22:09:05,682: INFO: common] created directory at: artifacts/data_ingestion


PermissionError: [WinError 5] Access is denied: 'artifacts/data_ingestion\\PetImages/Cat/'